# SAM3 Tracker - Google Colab

このノートブックでは、SAM3を使用したトラッキングベースのアノテーションをGoogle Colab上で実行します。

## ワークフロー
1. 依存関係のインストール
2. リポジトリのクローンとインポート
3. 画像のアップロード
4. SAM3 Trackerの初期化
5. オブジェクト選択（スライダーUI）
6. トラッキング実行
7. 結果の可視化とダウンロード

## Step 1: Setup - 依存関係のインストール

In [ ]:
# @title 1. 依存関係のインストール
# このセルを最初に実行してください

!pip install -q torch torchvision
!pip install -q opencv-python-headless
!pip install -q tqdm pyyaml

# SAM3のインストール（Facebook Research）
# 注意: SAM3の公式リポジトリURLを確認してください
# !pip install -q git+https://github.com/facebookresearch/sam3.git

print("Setup complete!")

## Step 2: Clone & Import - リポジトリの取得

In [ ]:
# @title 2. リポジトリのクローンとインポート
import sys

# リポジトリのURL（必要に応じて変更してください）
REPO_URL = "https://github.com/YOUR_USERNAME/hsr-perception-robocup.git"  # @param {type:"string"}
REPO_PATH = "/content/repo"

# クローン（既にクローン済みの場合はスキップ）
!git clone {REPO_URL} {REPO_PATH} 2>/dev/null || echo "Already cloned or clone failed - checking if exists..."

# パスを確認
import os
if os.path.exists(f"{REPO_PATH}/scripts/annotation"):
    sys.path.insert(0, f"{REPO_PATH}/scripts/annotation")
    print(f"Added to path: {REPO_PATH}/scripts/annotation")
else:
    print(f"Warning: {REPO_PATH}/scripts/annotation not found")
    print("Please check the repository URL or upload scripts manually.")

In [ ]:
# @title 2b. モジュールのインポート
try:
    from sam3_tracker import SAM3Tracker, SAM3TrackerConfig, ColabObjectSelector
    from colab_utils import upload_zip_and_extract, download_results, list_image_files
    from annotation_utils import AnnotationResult
    print("Import successful!")
except ImportError as e:
    print(f"Import error: {e}")
    print("\nPlease ensure the repository is cloned correctly.")

## Step 3: Upload Images - 画像のアップロード

In [ ]:
# @title 3. 画像のアップロード（zip形式）
# 画像をzip形式でまとめてアップロードしてください

INPUT_DIR = upload_zip_and_extract("/content/input_images")

# 画像一覧を表示
images = list_image_files(INPUT_DIR)
print(f"\nFound {len(images)} images:")
for img in images[:5]:
    print(f"  - {img}")
if len(images) > 5:
    print(f"  ... and {len(images) - 5} more")

## Step 4: Initialize - SAM3 Trackerの初期化

In [ ]:
# @title 4. SAM3 Trackerの初期化

# 設定パラメータ
GPU_ID = 0  # @param {type:"integer"}
MAX_LONG_SIDE = 1024  # @param {type:"integer"}
MIN_MASK_AREA = 500  # @param {type:"integer"}

config = SAM3TrackerConfig(
    gpu_id=GPU_ID,
    max_long_side=MAX_LONG_SIDE,
    min_mask_area=MIN_MASK_AREA,
    mask_threshold=0.5,
    box_margin=0.02,
)

tracker = SAM3Tracker(config)

print("Tracker initialized!")
print(f"  - GPU ID: {config.gpu_id}")
print(f"  - Max image size: {config.max_long_side}")
print(f"  - Min mask area: {config.min_mask_area}")

## Step 5: Select Object - オブジェクト選択

スライダーを使用して、トラッキング対象のオブジェクトの中心位置を指定してください。

1. X/Yスライダーを動かしてカーソル位置を調整
2. 緑の十字カーソルがオブジェクトの中心に来るように調整
3. 「Confirm Selection」ボタンをクリック

In [ ]:
# @title 5. オブジェクト選択（スライダーUI）

# 最初の画像を取得
first_image = list_image_files(INPUT_DIR)[0]
print(f"First frame: {first_image}")

# セレクターを作成して表示
selector = ColabObjectSelector(first_image)
selector.display()

In [ ]:
# @title 6. 選択の確認
# 「Confirm Selection」をクリックした後にこのセルを実行してください

click_point = selector.get_selection()

if click_point is None:
    print("ERROR: 選択が完了していません。")
    print("上のセルでスライダーを調整し、'Confirm Selection'をクリックしてください。")
else:
    print(f"Selected point: ({click_point[0]:.3f}, {click_point[1]:.3f})")
    print("Ready to run annotation!")

## Step 7: Annotate - トラッキング実行

In [ ]:
# @title 7. トラッキングアノテーション実行

import os

# 設定
CLASS_ID = 0  # @param {type:"integer"}
OUTPUT_DIR = "/content/output"  # @param {type:"string"}
SAVE_MASKS = True  # @param {type:"boolean"}
SAVE_CUTOUTS = True  # @param {type:"boolean"}

# 選択確認
if click_point is None:
    raise ValueError("オブジェクトが選択されていません。Step 5-6を完了してください。")

# 出力ディレクトリ作成
os.makedirs(OUTPUT_DIR, exist_ok=True)

# プログレスコールバック
def progress_callback(current, total):
    if current % 10 == 0 or current == total:
        print(f"Progress: {current}/{total} ({100*current//total}%)")

# アノテーション実行
print("Starting annotation...")
print(f"  - Input: {INPUT_DIR}")
print(f"  - Output: {OUTPUT_DIR}")
print(f"  - Class ID: {CLASS_ID}")
print(f"  - Click point: {click_point}")

try:
    result = tracker.annotate_sequence(
        input_dir=INPUT_DIR,
        class_id=CLASS_ID,
        output_dir=OUTPUT_DIR,
        click_point=click_point,
        save_masks=SAVE_MASKS,
        save_cutouts=SAVE_CUTOUTS,
        progress_callback=progress_callback,
    )

    print("\n" + "=" * 50)
    print(result.summary())

    if result.failed > 0:
        print("\nFailed images:")
        for path in result.failed_paths[:5]:
            print(f"  - {path}")

finally:
    tracker.shutdown()

## Step 8: Visualize - 結果の可視化

In [ ]:
# @title 8. 結果の可視化

import matplotlib.pyplot as plt
import cv2
from pathlib import Path

output_path = Path(OUTPUT_DIR)
images_dir = output_path / "images"
labels_dir = output_path / "labels"

# サンプル画像を取得
sample_images = sorted(images_dir.glob("*.jpg"))[:4]
if not sample_images:
    sample_images = sorted(images_dir.glob("*.png"))[:4]

if not sample_images:
    print("No annotated images found.")
else:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    axes = axes.flatten()

    for idx, img_path in enumerate(sample_images):
        # 画像読み込み
        img = cv2.imread(str(img_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w = img.shape[:2]

        # ラベル読み込み
        label_path = labels_dir / f"{img_path.stem}.txt"
        if label_path.exists():
            with open(label_path, "r") as f:
                line = f.readline().strip()
                parts = line.split()
                if len(parts) >= 5:
                    class_id, x_c, y_c, bw, bh = map(float, parts[:5])

                    # ピクセル座標に変換
                    x1 = int((x_c - bw / 2) * w)
                    y1 = int((y_c - bh / 2) * h)
                    x2 = int((x_c + bw / 2) * w)
                    y2 = int((y_c + bh / 2) * h)

                    # バウンディングボックス描画
                    cv2.rectangle(img_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)

        axes[idx].imshow(img_rgb)
        axes[idx].set_title(img_path.name)
        axes[idx].axis("off")

    # 残りの軸を非表示
    for idx in range(len(sample_images), 4):
        axes[idx].axis("off")

    plt.tight_layout()
    plt.show()

## Step 9: Download - 結果のダウンロード

In [ ]:
# @title 9. 結果のダウンロード

download_results(OUTPUT_DIR, "sam3_annotations.zip")
print("Download complete! Check your browser downloads.")

---

## Troubleshooting

### SAM3のインストールに失敗する場合
SAM3の公式リポジトリURLを確認し、Step 1のインストールコマンドを更新してください。

### メモリ不足エラー
- `MAX_LONG_SIDE`を小さくしてください（例: 512）
- ランタイムをリセットしてGPUメモリを解放してください

### 選択が反映されない
- 「Confirm Selection」ボタンをクリックしてから、次のセルを実行してください
- セレクターを再作成する場合は、Step 5のセルを再実行してください